In [1]:
from datetime import datetime, timedelta
import geojson
import geopandas as gpd
import numpy as np
import os
from pathlib import Path
import pandas as pd
from pyproj import Transformer, CRS
import rasterio as rio
from rasterio.mask import mask
from shapely.geometry import Polygon
from shapely.geometry import shape
import shutil
import subprocess
import time

# IMPORT FUNCTIONS --------------------------------------------------------
from ff_file_generator_automatic import create_ff_file
from ffgeojsonTogeojson import process_ffgeojson_files

# Import the relevant functions from extend.py
from extend import get_utm_crs, process_raster_files

# Import the creation of the cfg file for windNinja
from genWindNinjaFile_automatic import create_config_file
# -------------------------------------------------------------------------


# .........................
start_time = time.time()# .
# .........................


In [2]:
# Parent directory where all the subfolders are located
DEM_path = '/home/jsoma/europeData/EU-DEM.tif'
CLC_path = '/home/jsoma/europeData/EU-CLC_v2018.tif'
parent_dir = Path('/home/jsoma/examples/simulation_crete')


# Iterate through each subdirectory in the parent directory
for subdir in parent_dir.iterdir():

    print(f"Processing folder: {subdir}")
    run_dir = str(subdir)

    
    
    # Change to the folder
    os.chdir(subdir)
    # Find the file that ends with '_data.csv'
    wind_data_file = next((file for file in os.listdir('.') if file.endswith('_data.csv')), None)
    wind_data = pd.read_csv(wind_data_file)
    num_steps = np.shape(wind_data)[0]

    # Parameters -------------------------------------------------------
    # Get the common data
    common_data = wind_data.iloc[[0]]
    lat = common_data['lat'].iloc[0]
    lon = common_data['lon'].iloc[0]
    print(lat)
    size_km1 = 90  # Size of the square in kilometers
    size_km2 = 50

    # Get the UTM CRS
    dstCrs_UCTM = get_utm_crs(lon, lat)
    output_dir = os.getcwd()
    elevation_path, landcover_path = process_raster_files(lon, lat, size_km1, size_km2, DEM_path, CLC_path, output_dir, dstCrs_UCTM)

    # loop over the rows of wind_data to run the simulation
    # for i in range(num_steps):
    for i, row in wind_data.iterrows():

        folderName = f't{i}'

        if not os.path.exists(folderName):
            os.mkdir(folderName)
            shutil.copy('elevation.tif', folderName)  # Copy elevation file for all folders

            if i == 0:
                shutil.copy('land_cover.tif', folderName)  # Copy land_cover.tif only for t0
            else: 
                # Modify land cover based on previous step's GeoJSON output
                print(f"Attempting to read file: {geojson_file}")
                land_cover = os.path.join(subdir, 'land_cover.tif')
                out_forefire_geojson = gpd.read_file(geojson_file)

                with rio.open(land_cover) as src:
                    land_cover_array = src.read(1)
                    land_cover_array_copy = land_cover_array.copy()
                    land_cover_crs = src.crs

                    if out_forefire_geojson.crs != land_cover_crs:
                        out_forefire_geojson = out_forefire_geojson.to_crs(land_cover_crs)

                    geometries = [geom for geom in out_forefire_geojson.geometry]
                    out_image, out_transform = mask(src, geometries, crop=False, nodata=-1, filled=True)

                    land_cover_array_copy[out_image[0] != -1] = 324  # Set burned areas to specific value
                    land_cover_array_copy = land_cover_array_copy.astype(np.int16)

                    out_meta = src.meta.copy()

                    # Save modified land cover for the next iteration
                    output_path = os.path.join(folderName, 'land_cover.tif')
                    with rio.open(output_path, 'w', **out_meta) as dest:
                        dest.write(land_cover_array_copy, 1)
    
        
        os.chdir(folderName)
        print('-------------------')
        print(folderName)
        print('-------------------')
        # Store teh variables for this time-step
        wind_speed =  round(wind_data['wind_speed'][i], 1)
        wind_direction = round(wind_data['wind_direction'][i])
        date =  wind_data['date'][i]

        # 3) WindNinja Simulation  --------------------
        # Gen cfg file
        create_config_file(wind_speed, wind_direction)
        # run WindNinja
        subprocess.run(['WindNinja_cli', 'wind_domain_avg.cfg'])

        # 4) Create landscape.nc file once WindNinja is done --------
        subprocess.run(['python3', '/home/jsoma/scripts/scripts/allTogether/createLandscape.py'])

        # 5) Create a tX.ff file -------------------------------------
        
        create_ff_file(i, folderName, run_dir, date)
        
        # 6) Run ForeFire -------------------------------------------
        subprocess.run(['forefire', '-i', f't{i}.ff'], check=True)

        # 7) Proccess the output of ForeFire -----------------------
        process_ffgeojson_files()
        geojson_file = os.path.join(run_dir, f't{i}', f't{i}.geojson')

        # 8) Prepare for the next iteration ---------------------------
        os.chdir(subdir)
        if i == num_steps - 1:
            break
    


    # Change back to the parent directory
    os.chdir(parent_dir)

# Record the end time
end_time = time.time()

# Calculate and print the elapsed time
elapsed_time = end_time - start_time
print(f"Total elapsed time: {elapsed_time / 60:.2f} minutes")

Processing folder: /home/jsoma/examples/simulation_crete/2012-08-29_25.31E_35.26N
35.2593
UTM Zone: 35, Hemisphere: N, EPSG Code: EPSG:32635
-------------------
t0
-------------------
Configuration file 'wind_domain_avg.cfg' has been created successfully.
Run 0: Reading elevation file...
Run 0: Run number 0 started with 8 threads.
Run 0: Generating mesh...
Run 0: Initializing flow...
Run 0: Building equations...
Run 0: Solving...
Run 0 (solver): 13% complete
Run 0 (solver): 31% complete
Run 0 (solver): 59% complete
Run 0 (solver): 80% complete
Run 0 (solver): 91% complete
Run 0 (solver): 98% complete
Run 0 (solver): 100% complete
Run 0: Writing output files...
Run 0: Meshing time was 0.067926 seconds.
Run 0: Initialization time was 0.158330 seconds.
Run 0: Equation building time was 1.158149 seconds.
Run 0: Solver time was 9.876600 seconds.
Run 0: Output writing time was 0.056017 seconds.
Run 0: Total simulation time was 11.894070 seconds.
Run 0: Run number 0 done!
EPSG: 32635
writing 